In [1]:
import pandas as pd
import numpy
import time

In [44]:
#
# https://www.dataquest.io/blog/pandas-big-data/
#
def mem_usage(pandas_obj):
    if isinstance(pandas_obj,pd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep=True).sum()
    else: # we assume if not a df it's a series
        usage_b = pandas_obj.memory_usage(deep=True)
    usage_mb = usage_b / 1024 ** 2 # convert bytes to megabytes
    return "{:03.3f} MB".format(usage_mb)

In [65]:
dx = pd.read_csv('/home/starq/REP/DATA/FINANCE/Quotazioni/NVDA.csv' \
                 ,usecols=['Date','Open', 'High', 'Low', 'Close', 'Volume'] \
                 ,infer_datetime_format=True, parse_dates=True \
                 ,index_col='Date'
                )

dx.info(memory_usage='deep')


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4419 entries, 1999-01-22 to 2016-08-12
Data columns (total 5 columns):
Open      4419 non-null float64
High      4419 non-null float64
Low       4419 non-null float64
Close     4419 non-null float64
Volume    4419 non-null int64
dtypes: float64(4), int64(1)
memory usage: 207.1 KB


In [62]:
print(mem_usage(dx))

0.202 MB


In [76]:
gl = dx.copy()
dtypes = gl.drop(gl.index,axis=0).dtypes
#dtypes=gl.dtypes
dtypes_col = dtypes.index
dtypes_type = [i.name for i in dtypes.values]
dtypes, dtypes_col, dtypes_type
#pd.__version__
column_types = dict(zip(dtypes_col, dtypes_type))

preview = first2pairs = {key:value for key,value in list(column_types.items())}
import pprint
pp = pp = pprint.PrettyPrinter(indent=4)
pp.pprint(preview)

{   'Close': 'float64',
    'High': 'float64',
    'Low': 'float64',
    'Open': 'float64',
    'Volume': 'int64'}


In [74]:
read_and_optimized = pd.read_csv('/home/starq/REP/DATA/FINANCE/Quotazioni/NVDA.csv' \
                                 ,usecols=['Date','Open', 'High', 'Low', 'Close', 'Volume'] \
                                 ,dtype=column_types,parse_dates=['Date'],infer_datetime_format=True \
                                 ,index_col='Date')
print(mem_usage(read_and_optimized))

0.202 MB


In [75]:
read_and_optimized.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4419 entries, 1999-01-22 to 2016-08-12
Data columns (total 5 columns):
Open      4419 non-null float64
High      4419 non-null float64
Low       4419 non-null float64
Close     4419 non-null float64
Volume    4419 non-null int64
dtypes: float64(4), int64(1)
memory usage: 207.1 KB


### Closure x somma volumi 

In [3]:
# OK
#
def cum_vol(idx, vol):
    '''
    TEST : OK 

    in  : idx=timestamp, vol
    out : [idx, vc]
    '''
    _v = vol
    vc = vol
    idx  = idx
    #print('cum_vol : vol = {} - vc = {}'.format(vol,vc))
    
    def _compute(v): #, ask=False):
        
        nonlocal vc
        nonlocal idx
           
        vc += v
#        if ask:
#            return [idx, vc]
        return [idx, vc]
        
    return _compute

In [6]:
# da rivedere

def range_volatility(periods, idx, h, l):
    '''
    TEST : OK 

    in  : idx=timestamp, h, l
    out : [idx, ]
    '''
    h = h
    l = l
    idx = idx
    periods = periods
    counter = periods
    stack = list()
    stack.append({idx:{}})
    #print('cum_vol : vol = {} - vc = {}'.format(vol,vc))
    
    def _compute(h,l):
        
        nonlocal periods, stack, idx
        
        if len(stack) < periods:
            stack.append({idx:{}})
            '''
            TBD
            per ogni elemento dello stack:
                -aggiungo una quantità (es. volume)
                -sostituisco uno o più valori ohlc (includo/escludo secondo criteri)
            '''
            return None
        else:
            stack.append({idx:{}})
            '''rimuove il primo elemento dello stack'''
            ret = stack.pop(0)
            '''aggiunge al primo elemento dello stack il risultato del calcolo relativo'''
            ret['value'] = ret.keys() # tbd
            return ret

        #if counter == 0:
        #    stack.append({idx:{}})
        #    counter -= 1
        #    return None
        
        #print(idx)
        #return idx
        
    return _compute

In [ ]:
'''
volatilità relativa ad n periodi

- stack = []
- append(stack, valore)
- if len(stack) = n:
     - calcola la volatilità relativa dei valori sullo stack
     - rimuove il primo elemento dello stack (il meno recente)

'''

def range_volatility(periods, idx, h, l):
    '''
    TEST : OK 

    in  : idx=timestamp, h, l
    out : [idx, ]
    '''
    h = h
    l = l
    idx = idx
    periods = periods
    counter = periods
    stack = list()
    stack.append({idx:{}})
    #print('cum_vol : vol = {} - vc = {}'.format(vol,vc))
    
    def _compute(h,l):
        
        nonlocal periods, stack, idx
        
        stack.append({idx:{}})
        
        if len(stack) = periods:
            #calcola la volatilità relativa dei valori sullo stack
            # computed_result = f(x)
            #rimuove il primo elemento dello stack (il meno recente)
            ret = stack.pop(0)

            return computed_result
     
    return _compute

In [9]:
#
# OK
#

periods  = 5 ### weekly
vc_stack = []
rv_stack = []
vc       = None

### parametrizzare
loop     = periods - 1
rv_range = 50
###

start = time.clock()

rows = 0
for row in dx.itertuples(name=None): 
    
    ohlcv = list(row)             ### best performance 
    idx=ohlcv[0]
    v = ohlcv[5]
    h = ohlcv[2]
    l = ohlcv[3]

    if rows :
        if loop:
            vc = _cum_vol(v)
            print('in-loop = {} - v = {} - vc = {}'.format(loop,v, vc))
            loop -= 1
        else:
            #print('out-loop = {} - vc = {}'.format(loop,vc))
            vc_stack.append(vc)
            _cum_vol = cum_vol(idx, v)
            loop = periods - 1
            
        rv = _range_v(h,l)
        if rv: 
            rv_stack.append(rv)
            _range_v = range_volatility(rv_range, idx, h, l)        
            
    else:
        _cum_vol = cum_vol(idx, v) 
        _range_v = range_volatility(rv_range, idx, h, l)
        
    rows += 1
    
vc_stack.append(vc)
    
elapsed = str(time.clock() - start)

#print (vc_stack)
#print (*vc_stack, sep = "\n")
print (*rv_stack, sep = "\n")
# key = lambda x: float(x[1]) per ordinare in base a %gap
#print (*sorted(gap_stack, key = lambda x: float(x[1])), sep = "\n")
print ('\ntotal datapoint : {}'.format(rows))
print ('\ntotal rv items : {}'.format(len(rv_stack)))
print('elapsed : ' + elapsed)

        #row_list.append(dict( (a, active_indicators[a] for a in active_indicators)))

#row_list                    
    #startTime = time.perf_counter()
#    dict1 = dict( (a, active_indicators[a] for a in active_indicators)

in-loop = 4 - v = 12762000 - vc = [Timestamp('1999-01-22 00:00:00'), 80629200]
in-loop = 3 - v = 8580000 - vc = [Timestamp('1999-01-22 00:00:00'), 89209200]
in-loop = 2 - v = 6109200 - vc = [Timestamp('1999-01-22 00:00:00'), 95318400]
in-loop = 1 - v = 5688000 - vc = [Timestamp('1999-01-22 00:00:00'), 101006400]
in-loop = 4 - v = 3867600 - vc = [Timestamp('1999-01-29 00:00:00'), 9968400]
in-loop = 3 - v = 6602400 - vc = [Timestamp('1999-01-29 00:00:00'), 16570800]
in-loop = 2 - v = 1878000 - vc = [Timestamp('1999-01-29 00:00:00'), 18448800]
in-loop = 1 - v = 4548000 - vc = [Timestamp('1999-01-29 00:00:00'), 22996800]
in-loop = 4 - v = 3852000 - vc = [Timestamp('1999-02-05 00:00:00'), 7273200]
in-loop = 3 - v = 2174400 - vc = [Timestamp('1999-02-05 00:00:00'), 9447600]
in-loop = 2 - v = 3705600 - vc = [Timestamp('1999-02-05 00:00:00'), 13153200]
in-loop = 1 - v = 3306000 - vc = [Timestamp('1999-02-05 00:00:00'), 16459200]
in-loop = 4 - v = 5275200 - vc = [Timestamp('1999-02-12 00:00:00'

In [55]:
max_duration = 0
ai = dict()
ai['foo']= {'name':'foo_ind', 'freq':2}
ai['bar']= {'name':'bar_ind', 'freq':12}

#_freq        = {_:value['freq'] for (_,value) in ai.items()}
#max_duration = max([freq for freq in _freq.values()])
#max_duration = max(_freq.values())

max_duration = max({_:value['freq'] for (_,value) in ai.items()}.values())   ### one row OK



In [56]:
max_duration

12

In [9]:
i_stack = [[1,2,3,4,5],[2,2,2,2,2],[3,3,3,3,3]]
i_stack
i_stack[-2:]
#i_stack [[-2:], [1,2] ]
#v = fx['callback'] (i_stack[-fx['frequency':]])

[[2, 2, 2, 2, 2], [3, 3, 3, 3, 3]]

In [10]:
import numpy as np
a = np.arange(10).reshape(2, 5)

In [11]:
a

array([[0, 1, 2, 3, 4],
       [5, 6, 7, 8, 9]])

In [13]:
ixgrid = np.ix_([0,1], [2,4])
a[ixgrid]

array([[2, 4],
       [7, 9]])